# Perform decoding and encoding
Decode ROI and continuous images into text and encode text into continuous image.

NOTE: Continuous image decoding still has some problems.

In [1]:
%matplotlib inline
import nibabel as nib
from nltools.mask import create_sphere

from gclda.model import Model
from gclda.decode import Decoder
from gclda.utils import plot_brain

/Users/tsalo/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
%%time
# Load example model and initialize decoder
model = Model.load('gclda_results/20170728/model.pkl')
decoder = Decoder(model)

model.display_model_summary()

--- Model Summary ---
 Current State:
	 Current Iteration   = 1000
	 Initialization Seed = 1
	 Current Log-Likely  = -15432667.2492
 Model Hyper-Parameters:
	 Symmetric = True
	 n_topics  = 400
	 n_regions = 2
	 alpha     = 0.100
	 beta      = 0.010
	 gamma     = 0.010
	 delta     = 1.000
	 roi_size  = 50.000
	 dobs      = 25.0
 Model Training-Data Information:
	 Dataset Label                 = 20170728
	 Word-Tokens (n_word_tokens)   = 1033977
	 Peak-Tokens (n_peak_tokens)   = 413292
	 Word-Types (n_word_labels)    = 2476
	 Documents (n_docs)            = 11404
	 Peak-Dimensions (n_peak_dims) = 3
CPU times: user 39.1 s, sys: 2.32 s, total: 41.5 s
Wall time: 45.5 s


## Decode a region of interest
### From a file

In [3]:
%%time
roi_file = 'data/fusiform.nii.gz'
fusiform_df = decoder.decode_roi(roi_file)
fusiform_df = fusiform_df.sort_values(by='Weight', ascending=False)
print(fusiform_df.head(10))

            Weight
Term              
visual    0.027265
face      0.023594
faces     0.015642
fusiform  0.015329
memory    0.013289
stimuli   0.012839
task      0.012475
motor     0.011144
using     0.009932
object    0.009326
CPU times: user 23 s, sys: 28.5 s, total: 51.5 s
Wall time: 50.8 s


### From a set of coordinates

In [4]:
%%time
# Peak coordinates from Neurosynth fusiform meta-analysis.
coords = [[-40, -50, -16],
          [40, -50, -18]]
radii = [3] * len(coords)

# Create mask image
mask_data = (model.dataset.masker.volume.get_data()!=0).astype(int)
affine = model.dataset.masker.volume.affine
mask = nib.Nifti1Image(mask_data, affine)

roi_img = create_sphere(coords, radius=radii, mask=mask)
fusiform_df = decoder.decode_roi(roi_img)
fusiform_df = fusiform_df.sort_values(by='Weight', ascending=False)
print(fusiform_df.head(10))

NameError: name 'nib' is not defined

## Decode a continuous image
For some reason, the seemingly relevant features actually have the lowest weights, which is why `ascending` is currently set to `True`. We need to look into why this is happening. Also, when we decode `pFgA`, `pAgF`, `pFgA_given_pF=0.50`, or `pAgF_given_pF=0.50` images, we don't even get that pattern.

In [ ]:
%%time
continuous_file = 'data/faces_specificity_z.nii.gz'
faces_df = decoder.decode_continuous(continuous_file)
faces_df = faces_df.sort_values(by='Weight', ascending=True)
print(faces_df.head(10))

## Encode text into an image

In [ ]:
%%time
text = 'painful stimulation during a language task'
text_image = decoder.encode(text)
underlay = model.dataset.masker.volume
fig = plot_brain(text_image, underlay)